In [3]:
import math
from typing import Callable, List, Tuple, Optional

# -----------------------------
# Pagalbiniai: spausdinimas ir kriterijai
# -----------------------------
def _conv_ok(x_new: float, x_old: float, fx_new: float, eps: float) -> bool:
    """Jungtinis stabdymo kriterijus: aukštis + plotis."""
    height_ok = abs(fx_new) < eps
    width_ok  = abs(x_new - x_old) / max(1.0, abs(x_new)) < eps
    return height_ok and width_ok

def _print_result(tag: str, x: float, fx: float, iters: int, converged: bool, extra: str = ""):
    msg = f"[{tag}] x = {x:.16g}, f(x) = {fx:.3e}, iters = {iters}, converged = {converged}"
    if extra:
        msg += f", {extra}"
    print(msg)

# ============================================================
# 0) ŠAKNIES INTERVALO RADIMAS (LOKALIZAVIMAS)
# ============================================================

# 0.a) SKENAVIMAS PER ŽINGSNĮ: sign change scanning
def scan_sign_changes(
    f: Callable[[float], float],
    x_min: float, x_max: float, step: float
) -> List[Tuple[float, float]]:
    """
    Grąžina sąrašą intervalų [a,b], kuriuose f(a)*f(b)<0.
    Pastaba: jei žingsnis per didelis, šaknys gali būti praleistos.
    """
    if step <= 0:
        raise ValueError("scan_sign_changes: step must be > 0")
    if x_min > x_max:
        x_min, x_max = x_max, x_min

    intervals = []
    x_left = x_min
    f_left = f(x_left)
    x = x_left + step

    while x <= x_max + 1e-15:
        f_right = f(x)
        if f_left == 0.0:
            intervals.append((x_left - step*0.5, x_left + step*0.5))
        elif f_left * f_right < 0:
            intervals.append((x - step, x))
        x_left, f_left = x, f_right
        x += step

    return intervals

# 0.b) INTERVALO SUSIAURINIMAS SMULKESNIU ŽINGSNIU
def refine_bracket_by_rescan(
    f: Callable[[float], float],
    a: float, b: float, start_step: float, min_step: float
) -> Tuple[float, float]:
    """
    Duotas pradinis [a,b]. Skenuojam mažėjančiu žingsniu, kol randam piktesnį [A,B]
    su ženklų kaita ir (B-A) ~ min_step (arba kol pasiekiam min_step).
    Jei f(a)*f(b) jau <0, tiesiog gali gražinti [a,b] arba dar pasmailinti.
    """
    if a > b:
        a, b = b, a
    step = start_step
    A, B = a, b
    while step >= min_step:
        segs = scan_sign_changes(f, A, B, step)
        if segs:
            # Imame pirmą rastą segmentą ir tęsiame su dar smulkesniu žingsniu
            A, B = segs[0]
            # sumažinam žingsnį
            step *= 0.5
        else:
            break
    return (A, B)

# 0.c) „GRUBUS ĮVERTIS“ Daugianariams – Cauchy bound
def poly_cauchy_upper_bound(coeffs: List[float]) -> float:
    """
    Cauchy upper bound šaknų moduliui:
      Jei p(x)=a_n x^n + ... + a_0 (a_n != 0), tai
      visoms šaknims |z| <= 1 + max_{0<=k<n} |a_k/a_n|.
    coeffs: nuo didžiausio laipsnio iki laisvojo (pvz., [a_n,...,a_0])
    """
    if not coeffs or coeffs[0] == 0:
        raise ValueError("Cauchy bound: leading coefficient must be nonzero.")
    a_n = abs(coeffs[0])
    if a_n == 0:
        raise ValueError("Cauchy bound: leading coefficient must be nonzero.")
    ratios = [abs(c)/a_n for c in coeffs[1:]]  # be a_n
    Rteig = 1.0 + (max(ratios) if ratios else 0.0)
    return (-1 * Rteig, Rteig)

def poly_signed_bounds_course_style(coeffs: List[float]):
    """
    Kursinis/Lagrange-tipo receptas, atskirai teigiamiems ir neigiamiems:
      Teigiamiems:  R_pos = 1 + |a_{n-1}/a_n|
      Neigiamiems:  R_neg = -[ 1 + (|a_{0}|/|a_n|)^{1/(n-1)} ]
    Pastaba: coeffs – [a_n, a_{n-1}, ..., a_1, a_0]
    """
    if not coeffs or coeffs[0] == 0:
        raise ValueError("signed bounds: leading coefficient must be nonzero.")
    a_n = abs(coeffs[0])
    n = len(coeffs) - 1
    if n <= 0:
        return (-0.0, 0.0)

    # teigiamų šaknų viršutinė riba (paprastas kursinis įvertis)
    R_pos = 1.0 + abs(coeffs[1]) / a_n

    # neigiamų šaknų apatinė riba: naudok a0 (laisvąjį narį) ir laipsnį 1/(n-1)
    if n >= 2:
        R_neg = -(1.0 + (abs(coeffs[-1]) / a_n) ** (1.0 / (n - 1)))
    else:
        # n=1 (linijinis) – vien tik poslinkis
        R_neg = -(1.0 + abs(coeffs[-1]) / a_n)

    return (R_neg, R_pos)

# ============================================================
# 1) PUSIAUKIRTĖ (Bisection) – apskliaustas, patikimas
# ============================================================
def bisection(
    f: Callable[[float], float],
    a: float, b: float, eps: float = 1e-8, max_iter: int = 200, verbose: bool = True
) -> Tuple[float, int, bool]:
    fa, fb = f(a), f(b)
    if fa == 0.0:
        _print_result("Bisection", a, 0.0, 0, True)
        return a, 0, True
    if fb == 0.0:
        _print_result("Bisection", b, 0.0, 0, True)
        return b, 0, True
    if fa * fb > 0:
        raise ValueError("Bisection: f(a) and f(b) must have opposite signs.")

    x_old = a
    x = a
    for k in range(1, max_iter+1):
        x = 0.5*(a + b)
        fx = f(x)
        if _conv_ok(x, x_old, fx, eps):
            if verbose: _print_result("Bisection", x, fx, k, True)
            return x, k, True
        if fa * fx <= 0:
            b, fb = x, fx
        else:
            a, fa = x, fx
        x_old = x

    if verbose: _print_result("Bisection", x, f(x), max_iter, False)
    return x, max_iter, False

# ============================================================
# 2) STYGŲ METODAS (Regula Falsi) – apskliaustas, greitesnis už pusiaukirtę
# ============================================================
def false_position(
    f: Callable[[float], float],
    a: float, b: float, eps: float = 1e-8, max_iter: int = 1, verbose: bool = True
) -> Tuple[float, int, bool]:
    fa, fb = f(a), f(b)
    if fa == 0.0:
        _print_result("FalsePosition", a, 0.0, 0, True)
        return a, 0, True
    if fb == 0.0:
        _print_result("FalsePosition", b, 0.0, 0, True)
        return b, 0, True
    if fa * fb > 0:
        raise ValueError("FalsePosition: f(a) and f(b) must have opposite signs.")

    x_old = a
    x = a
    for k in range(1, max_iter+1):
        # styga per (a,fa) ir (b,fb)
        x = b - fb*(b - a)/(fb - fa)
        fx = f(x)

        if _conv_ok(x, x_old, fx, eps):
            if verbose: _print_result("FalsePosition", x, fx, k, True)
            return x, k, True

        # palaikome apskliaudimą
        if fa * fx <= 0:
            b, fb = x, fx
        else:
            a, fa = x, fx

        x_old = x

    if verbose: _print_result("FalsePosition", x, f(x), max_iter, False)
    return x, max_iter, False

# ============================================================
# 3) PAPRASTŲJŲ ITERACIJŲ METODAS: x_{k+1} = x_k + (1/α) f(x_k)
# ============================================================
def simple_iteration(
    f: Callable[[float], float],
    x0: float, alpha: float, eps: float = 1e-8, max_iter: int = 200, verbose: bool = True
) -> Tuple[float, int, bool]:
    x_old = x0
    for k in range(1, max_iter+1):
        x = x_old + (1.0/alpha)*f(x_old)
        fx = f(x)
        if _conv_ok(x, x_old, fx, eps):
            if verbose: _print_result("SimpleIter", x, fx, k, True, extra=f"alpha={alpha}")
            return x, k, True
        x_old = x
    if verbose: _print_result("SimpleIter", x_old, f(x_old), max_iter, False, extra=f"alpha={alpha}")
    return x_old, max_iter, False

# ============================================================
# 4) NIUTONAS (su slopinimu β)
# ============================================================
def newton(
    f: Callable[[float], float],
    df: Callable[[float], float],
    x0: float, eps: float = 1e-8, max_iter: int = 100, beta: float = 1.0, verbose: bool = True
) -> Tuple[float, int, bool]:
    x_old = x0
    for k in range(1, max_iter+1):
        fx = f(x_old)
        dfx = df(x_old)
        if dfx == 0:
            if verbose: print("[Newton] f'(x)=0 – žingsnio daryti negalima.")
            return x_old, k-1, False
        x = x_old - beta*fx/dfx
        fx_new = f(x)
        if _conv_ok(x, x_old, fx_new, eps):
            if verbose: _print_result("Newton", x, fx_new, k, True, extra=f"beta={beta}")
            return x, k, True
        x_old = x
    if verbose: _print_result("Newton", x_old, f(x_old), max_iter, False, extra=f"beta={beta}")
    return x_old, max_iter, False


# ============================================================
# 4.b) HALLEY (Householder p=2) – Teiloro 2-o eilės aproksimacija
#      x_{k+1} = x_k - 2 f f' / (2 (f')^2 - f f'')
#      (trečios eilės konvergencija arti šaknies)
# ============================================================
def halley(
    f: Callable[[float], float],
    df: Callable[[float], float],
    d2f: Callable[[float], float],
    x0: float, eps: float = 1e-8, max_iter: int = 100, verbose: bool = True
) -> Tuple[float, int, bool]:
    x_old = x0
    for k in range(1, max_iter + 1):
        fx = f(x_old)
        dfx = df(x_old)
        d2fx = d2f(x_old)

        # Saugos patikros
        denom = 2.0 * (dfx * dfx) - fx * d2fx
        if dfx == 0.0 and denom == 0.0:
            if verbose:
                print("[Halley] f'(x)=0 ir vardiklis=0 – stabdymas.")
            return x_old, k - 1, False
        if denom == 0.0:
            # Kris į įprastą Niutoną kaip atsarginį variantą
            step = fx / dfx if dfx != 0.0 else 0.0
        else:
            step = (2.0 * fx * dfx) / denom

        x = x_old - step
        fx_new = f(x)

        if _conv_ok(x, x_old, fx_new, eps):
            if verbose: _print_result("Halley", x, fx_new, k, True)
            return x, k, True

        x_old = x

    if verbose: _print_result("Halley", x_old, f(x_old), max_iter, False)
    return x_old, max_iter, False


# ============================================================
# 5) KIRSTINIŲ (SECANT) – neapskliaustas, nereikia išvestinės
# ============================================================
def secant(
    f: Callable[[float], float],
    x0: float, x1: float, eps: float = 1e-8, max_iter: int = 100, verbose: bool = True
) -> Tuple[float, int, bool]:
    f0, f1 = f(x0), f(x1)
    x_old = x1
    x = x1
    for k in range(1, max_iter+1):
        denom = (f1 - f0)
        if denom == 0:
            if verbose: print("[Secant] Denominator zero – stabdoma.")
            return x1, k-1, False
        x = x1 - f1*(x1 - x0)/denom
        fx = f(x)
        if _conv_ok(x, x_old, fx, eps):
            if verbose: _print_result("Secant", x, fx, k, True)
            return x, k, True
        x0, f0, x1, f1 = x1, f1, x, fx
        x_old = x
    if verbose: _print_result("Secant", x, f(x), max_iter, False)
    return x, max_iter, False


# ============================================================
# 6) STANDARTINĖS FUNKCIJOS (SciPy / NumPy): brentq / newton / fsolve / poly_roots
# ============================================================
def solve_with_scipy(
    f: Callable[[float], float] | None,
    method: str,
    eps: float = 1e-8,
    bracket: Optional[Tuple[float, float]] = None,
    x0: Optional[float] = None,
    verbose: bool = True,
    *,
    coeffs: Optional[List[float]] = None,   # <— nauja: polinomo koeficientai (nuo a_n iki a_0)
    real_only: bool = False,                # <— nauja: ar filtruoti tik realiąsias šaknis
    real_tol: float = 1e-10                 # <— nauja: tolerancija Im daliai, kai real_only=True
):
    """
    method ∈ {'brentq','newton','fsolve','poly_roots'}
      - 'brentq'  – reikia bracket=(a,b) su f(a)*f(b)<0 (apskliausta)
      - 'newton'  – reikia x0; išvestinės nepaduodam → kirstinių variantas
      - 'fsolve'  – reikia x0; bendro profilio šaknų ieškiklis
      - 'poly_roots' – reikia coeffs=[a_n,...,a_0]; grąžina visų šaknų sąrašą
                       (pasirinktinai tik realiąsias, jei real_only=True)
    """
    if method == 'poly_roots':
        try:
            import numpy as _np
        except Exception as e:
            raise RuntimeError("NumPy nepasiekiama. Įdiekite numpy, kad naudotumėte 'poly_roots'.") from e
        if not coeffs or coeffs[0] == 0:
            raise ValueError("poly_roots: pateik koeficientus [a_n, ..., a_0] su a_n != 0.")
        r = _np.roots(_np.array(coeffs, dtype=float))  # kompleksinės šaknys
        if real_only:
            r = r[_np.abs(r.imag) <= real_tol].real     # filtruojam „beveik realias“
        roots_list = sorted(map(float, r)) if real_only else sorted(r, key=lambda z: (z.real, z.imag))
        if verbose:
            if real_only:
                print(f"[poly_roots] realios šaknys ({len(roots_list)}): {roots_list}")
            else:
                print(f"[poly_roots] šaknys (kompleksinės, {len(roots_list)}): {roots_list}")
        return roots_list

    # — Toliau – tavo buvę SciPy metodai —
    try:
        from scipy import optimize as _opt
    except Exception as e:
        raise RuntimeError("SciPy nepasiekiama. Įdiekite scipy arba naudokite vietinius metodus.") from e

    if method == 'brentq':
        if not bracket:
            raise ValueError("SciPy brentq: reikia bracket=(a,b).")
        a,b = bracket
        root = _opt.brentq(f, a, b, xtol=eps, rtol=eps, maxiter=200)
        if verbose: print(f"[SciPy brentq] x = {root:.16g}, f(x) = {f(root):.3e}")
        return root

    elif method == 'newton':
        if x0 is None:
            raise ValueError("SciPy newton: reikia x0.")
        root = _opt.newton(f, x0, tol=eps, maxiter=100)  # be df → kirstinių variantas
        if verbose: print(f"[SciPy newton] x = {root:.16g}, f(x) = {f(root):.3e}")
        return root

    elif method == 'fsolve':
        if x0 is None:
            raise ValueError("SciPy fsolve: reikia x0.")
        root = _opt.fsolve(f, x0, xtol=eps, maxfev=200)
        root = float(root[0])
        if verbose: print(f"[SciPy fsolve] x = {root:.16g}, f(x) = {f(root):.3e}")
        return root

    else:
        raise ValueError("method must be one of {'brentq','newton','fsolve','poly_roots'}")


# ============================================================
# PAVYZDINIS NAUDOJIMAS: susidek savo f, df, eps ir kviesk
# ============================================================
def main():
    # PAVYZDYS: x*math.cos(x) - 1.0
    def f(x: float) -> float:
        return 2.5 * x**4 - 2

    def df(x: float) -> float:
        return -5.16 * x**3 + 15.24 * x**2 - 5.52 * x - 6.31

    eps = 1e-8

    # ---- 0) ŠAKNIES INTERVALO RADIMAS ----
    # 0.a) SKENAVIMAS per žingsnį:
    # intervals = scan_sign_changes(f, x_min=-10.0, x_max=10.0, step=0.1)
    # print("Sign-change intervals:", intervals)

    # 0.b) INTERVALO SUSIAURINIMAS smulkesniu žingsniu:
    # A, B = refine_bracket_by_rescan(f, a=-10.0, b=10.0, start_step=0.1, min_step=1e-3)
    # print("Refined bracket:", (A, B))

    # 0.c) „GRUBUS ĮVERTIS“ (daugianariams) – Cauchy:
    # Pvz., p(x) = 2x^4 - 3x^3 + 0*x^2 + 5x - 7  → coeffs = [2, -3, 0, 5, -7]
    coeffs = [1, 1, -17, 17, 10]
    # print("Grubus ivertis:", poly_cauchy_upper_bound(coeffs))

    # 0.d) „TIKSLESNIS ĮVERTIS“ (daugianariams) – Fujiwara:
    print("Tikslesnis ivertis:", poly_signed_bounds_course_style(coeffs))

    # ---- 1) PUSIAUKIRTĖ (reikia atskyrimo) ----
    # bisection(f, a=-1.097513, b=-0.997513, eps=eps)

    # ---- 2) STYGOS / Regula Falsi (reikia atskyrimo) ----
    # false_position(f, a=0, b=1.0, eps=eps)

    # ---- 3) PAPRASTOS ITERACIJOS (parinkti α, kad |φ'|<1 ties šaknimi) ----
    # simple_iteration(f, x0=-1.05, alpha=5.0, eps=eps)

    # ---- 4) NIUTONAS (galima β<1 slopinimui) ----
    # newton(f, df, x0=-1.097513, eps=eps, beta=1.0)

    # ---- 4.b) HALLEY (reikia f', f'') ----
    # def d2f(x: float) -> float:
        # return -15.48 * x**2 + 30.48 * x - 5.52
    # halley(f, df, d2f, x0=-1.05, eps=eps, max_iter=50)

    # ---- 5) KIRSTINĖS (SECANT) – dvi pradinės reikšmės ----
    # secant(f, x0=-1.097513, x1=-0.997513, eps=eps)

    # ---- 6) STANDARTINĖS FUNKCIJOS (SciPy) ----
    # all_roots = solve_with_scipy(None, 'poly_roots', coeffs=coeffs, real_only=False, verbose=True)
    # real_roots = solve_with_scipy(None, 'poly_roots', coeffs=coeffs, real_only=True, verbose=True)
    # solve_with_scipy(f, method='brentq', eps=eps, bracket=(-1.097513, -0.997513))
    # solve_with_scipy(f, method='newton', eps=eps, x0=-1.097513)
    # solve_with_scipy(f, method='fsolve', eps=eps, x0=-1.097513)
    pass

main()

Tikslesnis ivertis: (-3.154434690031884, 2.0)
